# Yelp and Kaggle

In [1]:
import json
import pandas as pd
import numpy as np
import itertools

jsonfile_path = "business.json"

json_data = [json.loads(line) for line in open("business.json", encoding = "utf8")]
json_data[3]

{'business_id': 'xvX2CttrVhyG2z1dFg_0xw',
 'name': 'Farmers Insurance - Paul Lorenz',
 'address': '15655 W Roosevelt St, Ste 237',
 'city': 'Goodyear',
 'state': 'AZ',
 'postal_code': '85338',
 'latitude': 33.4556129678,
 'longitude': -112.3955963552,
 'stars': 5.0,
 'review_count': 3,
 'is_open': 1,
 'attributes': None,
 'categories': 'Insurance, Financial Services',
 'hours': {'Monday': '8:0-17:0',
  'Tuesday': '8:0-17:0',
  'Wednesday': '8:0-17:0',
  'Thursday': '8:0-17:0',
  'Friday': '8:0-17:0'}}

In [2]:
yelp_df=pd.DataFrame(json_data)
yelp_df.head(2)

,address,attributes,business_id,categories,city,hours,is_open,latitude,longitude,name,postal_code,review_count,stars,state
0,2818 E Camino Acequia Drive,{'GoodForKids': 'False'},1SWheh84yJXfytovILXOAQ,"Golf, Active Life",Phoenix,None,0,33.522143,-112.018481,Arizona Biltmore Golf Club,85016,5,3.0,AZ
1,30 Eglinton Avenue W,"{'RestaurantsReservations': 'True', 'GoodForMe...",QXAEGFB4oINsVuTFxEYKFQ,"Specialty Food, Restaurants, Dim Sum, Imported...",Mississauga,"{'Monday': '9:0-0:0', 'Tuesday': '9:0-0:0', 'W...",1,43.605499,-79.652289,Emerald Chinese Restaurant,L5R 3E7,128,2.5,ON


In [3]:
yelp_shortened_df =  yelp_df[['name','city', 'state', 'latitude', 'longitude', 'stars']].set_index('state')
yelp_ffplaces= yelp_shortened_df.loc[['NC']]
yelp_ffplaces.shape

(14720, 5)

# Top NC business, ct
### have not yet unified resturants with mispelt names

In [4]:
# groupNCbusiness=business_all_yelp.groupby('name')['business_id'].nunique()
# NCnames=groupNCbusiness.sort_values(ascending=False)
# NCnames.head()

In [5]:
# business_yelp_df=business_all_yelp[['name','city', 'latitude', 'longitude', 'stars']].set_index('name')
# yelp_fastfood = business_df.loc[["McDonald's"]]
# yelp_fastfood.sort_values('city').head()

# Adding Kagle

In [6]:
csv_path= "FastFoodKagle.csv"

kagle_df = pd.read_csv(csv_path)

In [7]:
kagle_df["stars"]=""
# kagle_df["province"]

kagle_ff_df=kagle_df.rename(columns={'province':'state'})

kagle_ff_df.head(1)

,address,city,country,keys,latitude,longitude,name,postalCode,state,websites,stars
0,324 Main St,Massena,US,us/ny/massena/324mainst/-1161002137,44.9213,-74.89021,McDonald's,13662,NY,"http://mcdonalds.com,http://www.mcdonalds.com/...",


In [8]:
shortened_kagle =  kagle_ff_df[['name', 'city', 'state', 'latitude', 'longitude', 'stars']].set_index('state')
kagle_ff= shortened_kagle.loc[['NC']]
kagle_ff.head()

,name,city,latitude,longitude,stars
state,,,,,
NC,Taco Bell,Leland,34.233890,-77.995440,
NC,McDonald's,Leland,34.211880,-78.024540,
NC,McDonald's,Leland,34.234477,-78.000649,
NC,McDonald's,Hickory,35.753320,-81.285720,
NC,Domino's Pizza,Fayetteville,34.929700,-78.856680,


In [9]:
NC_ff=kagle_ff.append(yelp_ffplaces)
NC_ff.shape

(15078, 5)

In [10]:
NC_ff_weird_coordinate_removal=NC_ff.sort_values(['longitude'])
NC_ff_weird_coordinate_removal.head()


,name,city,latitude,longitude,stars
state,,,,,
NC,American Answering Service,Las Vegas,36.105837,-115.117432,1
NC,Taco Bell,Oneonta,33.948653,-86.471480,
NC,McDonald's,Murphy,35.080654,-84.035812,
NC,Bojangles' Famous Chicken 'n Biscuits,Bryson City,35.425296,-83.446639,
NC,Arby's,Bryson City,35.422920,-83.446090,


In [11]:
NC_ff_weird_coordinate_removal= NC_ff_weird_coordinate_removal[NC_ff_weird_coordinate_removal.city !='Las Vegas']
NC_ff_weird_coordinate_removal=NC_ff_weird_coordinate_removal[NC_ff_weird_coordinate_removal.city !='Oneonta']
NC_ff = NC_ff_weird_coordinate_removal
NC_ff.head()

,name,city,latitude,longitude,stars
state,,,,,
NC,McDonald's,Murphy,35.080654,-84.035812,
NC,Bojangles' Famous Chicken 'n Biscuits,Bryson City,35.425296,-83.446639,
NC,Arby's,Bryson City,35.422920,-83.446090,
NC,Burger King,Franklin,35.165660,-83.400510,
NC,Bojangles' Famous Chicken 'n Biscuits,Franklin,35.162468,-83.392228,


In [12]:
NC_ff['new_name'] = NC_ff['name'].str.replace(r"[\"\',\s\.\-\&]", '').str.upper()
# NC_business_all.head(20)

# NC_business_all[NC_business_all['name'].str.contains('[Dd]onald|Bojangle')]
NC_10_ff=NC_ff[NC_ff['new_name'].str.contains('CHICKFIL|BOJANGLE|BURGERKING|SUBWAY|WENDYS|MCDONALD|TACOBELL|PIZZAHUT')]
ff_top_picks=NC_10_ff.drop_duplicates(subset=['new_name','latitude','longitude'], keep=False)
ff_top_picks.reset_index()

,state,name,city,latitude,longitude,stars,new_name
0,NC,McDonald's,Murphy,35.080654,-84.035812,,MCDONALDS
1,NC,Bojangles' Famous Chicken 'n Biscuits,Bryson City,35.425296,-83.446639,,BOJANGLESFAMOUSCHICKENNBISCUITS
2,NC,Burger King,Franklin,35.165660,-83.400510,,BURGERKING
3,NC,Bojangles' Famous Chicken 'n Biscuits,Franklin,35.162468,-83.392228,,BOJANGLESFAMOUSCHICKENNBISCUITS
4,NC,Wendy's,Franklin,35.160043,-83.391085,,WENDYS
5,NC,McDonald's,Franklin,35.186930,-83.371370,,MCDONALDS
6,NC,Wendy's,Cherokee,35.470750,-83.310643,,WENDYS
7,NC,Bojangles' Famous Chicken 'n Biscuits,Sylva,35.362077,-83.202253,,BOJANGLESFAMOUSCHICKENNBISCUITS
8,NC,Taco Bell,Waynesville,35.465950,-83.006950,,TACOBELL
9,NC,Chick-fil-A,Waynesville,35.503586,-82.989450,,CHICKFILA


In [13]:
ff_top_picks["new_name"]=ff_top_picks["new_name"].replace(
{'BOJANGLESCHICKENANDBISCUITS':'Bojangles', 'BOJANGLESCOLISEUM':'Bojangles','BOJANGLESFAMOUSCHICKENNBISCUITS':'Bojangles',
'BOJANGLESRESTAURANT':'Bojangles','CHICKFILA':'Chick-fil-A', 'MCDONALDGROUPWILKINSONANDASSOCIATESERA':'NA',
 'MCDONALDS':"McDonald's",'PIZZAHUT':'Pizza Hut', 'RONALDMCDONALDHOUSEOFCHARLOTTE':'NA','SUBWAY':"Subway",'WENDYS':"Wendy's",
"TACOBELL":'Taco Bell','BURGERKING':'Burger King','BURGERKINGRESTAURANT':'Burger King',"BOJANGLES":'Bojangles'})



In [14]:
ff_top_picks.to_json('fastgeodata.json', orient='records')

In [15]:
US_cities_df=pd.read_csv("uscitiesv1.5.csv")

US_cities_df=US_cities_df[['city','state_id','county_name']]
NC_cities_df=US_cities_df[US_cities_df["state_id"] == "NC"].set_index("state_id").reset_index()
NC_cities_df.head(10)

jumbo_df = NC_cities_df.set_index("city").join(ff_top_picks.set_index("city"), how = "right").reset_index()

jumbo_df.head()


,city,state_id,county_name,name,latitude,longitude,stars,new_name
0,Advance,NC,Davie,Bojangles' Famous Chicken 'n Biscuits,36.004400,-80.439700,,Bojangles
1,Ahoskie,NC,Hertford,Burger King,36.276270,-76.977450,,Burger King
2,Ahoskie,NC,Hertford,Wendy's,36.272189,-76.970790,,Wendy's
3,Albemarle,NC,Stanly,Subway,35.386855,-80.193661,,Subway
4,Albemarle,NC,Stanly,Taco Bell,35.347539,-80.166738,,Taco Bell


In [16]:
mountain_counties=['Alleghany','Ashe', 'Avery', 'Buncombe', 'Burke', 'Caldwell', 'Clay','Cherokee', 'Graham','Haywood',
                   'Henderson', 'Jackson','Macon','Madison','Mitchell','Polk','Swain', 'Transylvania','Watauga','Wilkes','Yancey']
coastal_counties=['Beaufort','Bertie','Bladen','Brunswick','Camden','Carteret','Chowan','Colombus','Craven','Duplin','Gates',
                  'Greene','Hertford','Jones','Lenoir','Martin','New Hanover','Onslow','Pamlico','Pasquotank','Pender',
                  'Perquimans','Pitt','Tyrell','Washington']

# #For actual name of region
# def county_typer(string):
#     if string in mountain_counties:
#         return("mountain")
#     elif string in coastal_counties:
#         return("coastal")
#     else:
#         return("central")


#for numerical conversion of region (mountain=1,piedmont=2,coastal=3)
def county_typer(string):
    if string in mountain_counties:
        return(1)
    elif string in coastal_counties:
        return(3)
    else:
        return(2)

In [17]:
jumbo_df["region"] = jumbo_df["county_name"].apply(func = county_typer)

jumbo_df.head(10)
stacked_area_df=jumbo_df

# Making geojson file

In [18]:
def df_to_geojson(df, properties, lat='latitude', lon='longitude'):
    geojson = {'type':'FeatureCollection', 'features':[]}
    for _, row in df.iterrows():
        feature = {'type':'Feature',
                   'properties':{},
                   'geometry':{'type':'Point',
                               'coordinates':[]}}
        feature['geometry']['coordinates'] = [row[lon],row[lat]]
        for prop in properties:
            feature['properties'][prop] = row[prop]
        geojson['features'].append(feature)
    return geojson

In [19]:
useful_columns = ['name','new_name', 'city', 'stars','region']
geojson_dict = df_to_geojson(jumbo_df, properties=useful_columns)
geojson_str = json.dumps(geojson_dict, indent=2)

In [20]:
# print(geojson_str)

In [21]:
output_filename = 'ff_geodataset.json'
with open(output_filename, 'w') as output_file:
    output_file.write('var dataset = {};'.format(geojson_str))
    
# how many features did we save to the geojson file?
# print('{} geotagged features saved to file'.format(len(geojson_dict['features']))

# CSV for stacked area

In [22]:
#df.to_csv(file_name, encoding='utf-8', index=False)

In [23]:
stacked_area_df.head()

,city,state_id,county_name,name,latitude,longitude,stars,new_name,region
0,Advance,NC,Davie,Bojangles' Famous Chicken 'n Biscuits,36.004400,-80.439700,,Bojangles,2
1,Ahoskie,NC,Hertford,Burger King,36.276270,-76.977450,,Burger King,3
2,Ahoskie,NC,Hertford,Wendy's,36.272189,-76.970790,,Wendy's,3
3,Albemarle,NC,Stanly,Subway,35.386855,-80.193661,,Subway,2
4,Albemarle,NC,Stanly,Taco Bell,35.347539,-80.166738,,Taco Bell,2


In [52]:
counts_df = stacked_area_df.groupby(['new_name','region']).count()['city'].reset_index()
counts_df = counts_df[counts_df['new_name']!= "NA"]
counts_df
# print(stacked_area_df.groupby(['region'])['new_name'].count())
# test=(stacked_area_df[['region','new_name']]).groupby(['region'])

,new_name,region,city
0,Bojangles,1,8
1,Bojangles,2,89
2,Bojangles,3,5
3,Burger King,1,3
4,Burger King,2,47
5,Burger King,3,7
6,Chick-fil-A,1,1
7,Chick-fil-A,2,42
8,Chick-fil-A,3,2
9,McDonald's,1,5


In [ ]:
counts_df['percent']=(counts_df['city']-counts_df['city'].min())/(counts_df['city']


In [57]:
wide_data = counts_df.pivot(index = "region", columns = "new_name")

In [28]:
# from collections import Counter

In [27]:
# graph_data['ff_ct']=[]
# graph_data=stacked_area_df[['new_name','region']]
# graph_data.to_dict('records')
    

KeyError: "['count'] not in index"

In [53]:
# counts_df.to_csv('area_test_data.csv', encoding= 'utf-8',index=False)

In [58]:
wide_data.to_csv("area_wide_data.csv", encoding='utf-8', index = False)